In [106]:
import pandas as pd
import json
import re
import sys
import os
import numpy as np
from pydub import AudioSegment
import tempfile

sys.path.append(os.path.abspath(".."))

def dummy_npwarn_decorator_factory():
  # https://stackoverflow.com/questions/77064579/module-numpy-has-no-attribute-no-nep50-warning SACADO DE AQUI
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)


from AUDIO.scripts.AudioEmotionAnalzer import AudioEmotionAnalysis
from TEXT.POO.scripts.TextEmotionAnalizer import TextEmotionClassifier

### Partimos de un csv enriquecido tras la primera clasificación de preguntas y respuestas y del json descargado de la API de Earningcall con los timestamps palabra a palabra.

In [107]:
df_csv = pd.read_csv('/home/aacastro/Alejandro/ACA_MultichanelAI_2025/src/TEXT/POO/annotated/AAPL/2018/Q3.csv')
with open('/home/aacastro/mchai/companies/AAPL/2018/Q3/LEVEL_3.json', 'r') as f:
    data_json = json.load(f)

# Conversión de intervenciones a frases con timestamps

A partir de un CSV con intervenciones completas y un JSON con las palabras y sus tiempos de inicio, generamos un nuevo DataFrame donde cada fila representa una frase individual.

Primero, se agrupan las palabras del JSON hasta formar frases completas, detectando los finales con signos de puntuación. Para cada frase se guarda el texto, el hablante, y los tiempos de inicio y fin.

Luego, se divide cada intervención del CSV en frases usando el mismo criterio. A cada frase se le copia toda la información original (speaker, título, sección, etc.).

Finalmente, se alinean las frases del CSV y del JSON por orden, y se asignan los tiempos de inicio y fin a cada frase. El resultado es un DataFrame enriquecido y preparado para trabajar a nivel de frase con contexto y tiempo.

In [108]:
# 2. Extraer frases del JSON con start y end time
frases_json = []

for speaker in data_json["speakers"]:
    words = speaker["words"]
    times = speaker["start_times"]
    speaker_name = speaker["speaker_info"]["name"]

    frase = ""
    tiempos = []

    for palabra, tiempo in zip(words, times):
        frase += palabra + " "
        tiempos.append(tiempo)

        if re.match(r".*[\.\!\?]$", palabra):  # fin de frase
            frases_json.append({
                "speaker": speaker_name,
                "text": frase.strip(),
                "start_time": tiempos[0],
                "end_time": tiempos[-1]
            })
            frase = ""
            tiempos = []

df_json = pd.DataFrame(frases_json)


# 3. Dividir cada intervención del CSV en frases y replicar metadatos
frases_expandidas = []

for _, row in df_csv.iterrows():
    frases = re.split(r'(?<=[\.\!\?])\s+', row["text"])  # frases del texto
    for frase in frases:
        if frase.strip():
            nueva_fila = row.to_dict()
            nueva_fila["text"] = frase.strip()  # frase individual
            frases_expandidas.append(nueva_fila)

df_expandido = pd.DataFrame(frases_expandidas)


# 4. Asignar start_time y end_time desde el JSON (asumiendo orden)
if len(df_expandido) != len(df_json):
    print(f"⚠️ Atención: Nº frases CSV ({len(df_expandido)}) != Nº frases JSON ({len(df_json)})")

# Añadir tiempos si hay correspondencia
min_len = min(len(df_expandido), len(df_json))
df_expandido = df_expandido.iloc[:min_len].copy()
df_json = df_json.iloc[:min_len].copy()

df_expandido["start_time"] = df_json["start_time"].values
df_expandido["end_time"] = df_json["end_time"].values

⚠️ Atención: Nº frases CSV (429) != Nº frases JSON (428)


In [109]:
# df_expandido.to_csv('sentences.csv')
df_expandido.head(20)

,speaker_id,name,title,text,start_time,end_time,Conf_Section,classification,global_confidence,Pair
0,spk05,Operator,Conference Call Operator,Please stand by.,0.765,1.385,prepared_remarks,Presentation,100.0,NaN
1,spk05,Operator,Conference Call Operator,We're about to begin.,1.626,2.206,prepared_remarks,Presentation,100.0,NaN
2,spk05,Operator,Conference Call Operator,"Good day, and welcome to the Apple Incorporate...",3.467,8.130,prepared_remarks,Presentation,100.0,NaN
3,spk05,Operator,Conference Call Operator,Today's call is being recorded.,8.931,9.931,prepared_remarks,Presentation,100.0,NaN
4,spk05,Operator,Conference Call Operator,"At this time, for opening remarks and introduc...",10.832,16.276,prepared_remarks,Presentation,100.0,NaN
5,spk05,Operator,Conference Call Operator,Please go ahead.,17.116,17.496,prepared_remarks,Presentation,100.0,NaN
6,spk06,Nancy Paxton,Senior Director of Investor Relations,Thank you.,19.838,20.078,prepared_remarks,Presentation,100.0,NaN
7,spk06,Nancy Paxton,Senior Director of Investor Relations,"Good afternoon, and thanks to everyone for joi...",20.298,22.480,prepared_remarks,Presentation,100.0,NaN
8,spk06,Nancy Paxton,Senior Director of Investor Relations,"Speaking first today is Apple CEO Tim Cook, an...",22.560,27.904,prepared_remarks,Presentation,100.0,NaN
9,spk06,Nancy Paxton,Senior Director of Investor Relations,"After that, we'll open the call to questions f...",29.238,31.320,prepared_remarks,Presentation,100.0,NaN


In [110]:
df_expandido.tail(20)

,speaker_id,name,title,text,start_time,end_time,Conf_Section,classification,global_confidence,Pair
408,spk03,Tim Cook,CEO,I see these things as being somewhat fluid and...,3515.506,3517.627,q_a,Answer,60.00,pair_8
409,spk06,Nancy Paxton,Senior Director of Investor Relations,So they're complementary.,3518.207,3518.547,q_a,Procedure,100.00,NaN
410,spk06,Nancy Paxton,Senior Director of Investor Relations,"Thank you, Laura.",3519.548,3520.108,q_a,Procedure,100.00,NaN
411,spk06,Nancy Paxton,Senior Director of Investor Relations,"Okay, that makes sense.",3520.308,3521.088,q_a,Procedure,100.00,NaN
412,spk06,Nancy Paxton,Senior Director of Investor Relations,"All right, thank you very much.",3521.868,3522.488,q_a,Procedure,100.00,NaN
413,spk03,Tim Cook,CEO,That's exactly right.,3523.089,3523.709,q_a,Procedure,66.67,NaN
414,spk03,Tim Cook,CEO,Thank you for the question.,3524.605,3524.765,q_a,Procedure,66.67,NaN
415,spk06,Nancy Paxton,Senior Director of Investor Relations,Thank you.,3525.045,3525.345,q_a,Procedure,100.00,NaN
416,spk06,Nancy Paxton,Senior Director of Investor Relations,"Thank you, Laura.",3525.645,3534.269,q_a,Procedure,100.00,NaN
417,spk06,Nancy Paxton,Senior Director of Investor Relations,A replay of today's call will be available for...,3534.950,3537.231,q_a,Procedure,100.00,NaN


### Aquí seleccionamos para procesar solo las frases que están dentro de intervenciones clasificadas como:

- Presentación (Previo a las preguntas y respuestas)
- Pregunta
- Respuesta

In [111]:
clasificaciones_deseadas = ['Presentation', 'Question', 'Answer']
df_filtrado = df_expandido[df_expandido['classification'].isin(clasificaciones_deseadas)]

In [112]:
for _, row in df_filtrado.head(5).iterrows():
    print(f"[{row['classification']}] {row['text']} ({row['start_time']}s) ({row['end_time']}s)")

[Presentation] Please stand by. (0.765s) (1.385s)
[Presentation] We're about to begin. (1.626s) (2.206s)
[Presentation] Good day, and welcome to the Apple Incorporated third quarter fiscal year 2018 earnings conference call. (3.467s) (8.13s)
[Presentation] Today's call is being recorded. (8.931s) (9.931s)
[Presentation] At this time, for opening remarks and introductions, I'd like to turn the call over to Nancy Paxton, Senior Director of Investor Relations. (10.832s) (16.276s)


### Orden de las emociones en los embeddings: ['happy', 'neutral', 'surprise', 'disgust', 'anger', 'sadness', 'fear']

## Text

In [113]:
frase = 'Good day, and welcome to the Apple Incorporated third quarter fiscal year 2018 earnings conference call.'

clf_text = TextEmotionClassifier()

text_embeddings = clf_text.get_embeddings(frase)
text_embeddings

Device set to use cuda:0


tensor([ 4.1922,  2.3097,  0.8478, -1.3362, -1.8390, -1.9948, -2.1797])

## Audio

In [114]:
def cortar_audio_temporal(input_path, start_time: int, end_time: int):
    """
    Recorta un archivo MP3 entre un rango de tiempo específico y devuelve un archivo temporal WAV.

    Args:
        input_file (str): Ruta del archivo MP3 de entrada.
        start_time (int): Tiempo inicial en segundos.
        end_time (int): Tiempo final en segundos.

    Returns:
        tempfile.NamedTemporaryFile: Archivo temporal WAV listo para usar.
    """
    try:
        # Cargar el audio
        audio = AudioSegment.from_mp3(input_path)

        # Convertir tiempos a milisegundos
        # start_ms = start_time * 1000
        # end_ms = end_time * 1000
        start_ms = int(start_time * 1000)
        end_ms = int((end_time + 0.25) * 1000)

        # Cortar el segmento
        segmento = audio[start_ms:end_ms]
        display(Audio(segmento.export(format="mp3").read(), rate=44100))

        # Crear archivo temporal WAV
        temp_wav = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
        segmento.export(temp_wav.name, format="wav")

        return temp_wav

    except Exception as e:
        print(f"Error al procesar el archivo: {e}")
        return None

In [115]:
audio_path = "/home/aacastro/mchai/RAVDESS/audios/Actor_01/03-01-05-02-01-01-01.wav"

clf_audio = AudioEmotionAnalysis()

audio_embeddings = clf_audio.get_embeddings(audio_path)
audio_embeddings

2025-03-30 18:27:11,108 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Detect model requirements, begin to install it: /home/aacastro/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/requirements.txt
install model requirements successfully
ckpt: /home/aacastro/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
init param, map: modality_encoders.AUDIO.extra_tokens from d2v_model.modality_encoders.AUDIO.extra_tokens in ckpt
init param, map: modality_encoders.AUDIO.alibi_scale from d2v_model.modality_encoders.AUDIO.alibi_scale in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias in ckpt
init param, map: modal

rtf_avg: 0.015: 100%|██████████| 1/1 [00:00<00:00, 15.77it/s]                                                                                      


tensor([-1.6415, -2.8577, -0.3029, -5.5051, 18.1174, -2.8723, -4.9379])

## Video


Para añadir la sección de video únicamente sería necesario el archivo de video alineado en tiempo con el archivo de audio grande de cada conferencia.

## Integration

In [117]:
input_path = "/home/aacastro/mchai/companies/AAPL/2018/Q3/audio.mp3"

# Itera sobre los primeros 5 segmentos
for i, (_, row) in enumerate(df_filtrado.head(5).iterrows(), start=1):
    print("=" * 70)
    print(f"🎧 Segmento {i}")
    print("-" * 70)

    temp_file = cortar_audio_temporal(input_path, row['start_time'], row['end_time'])

    # Mostrar información textual
    print(f"📌 Clasificación: {row['classification']}")
    print(f"🗣️  Texto: {row['text']}")
    print(f"⏱️  Intervalo de tiempo: {row['start_time']}s - {row['end_time']}s")

    # Obtener embeddings
    audio_embeddings = clf_audio.get_embeddings(temp_file.name)
    text_embeddings = clf_text.get_embeddings(row['text'])

    # Mostrar embeddings
    print("\n🔊 Embeddings de AUDIO:")
    print(audio_embeddings)
    print("\n📝 Embeddings de TEXTO:")
    print(text_embeddings)
    
    print("\n")  # Espacio entre segmentos
    os.remove(temp_file.name)

🎧 Segmento 1
----------------------------------------------------------------------


📌 Clasificación: Presentation
🗣️  Texto: Please stand by.
⏱️  Intervalo de tiempo: 0.765s - 1.385s


rtf_avg: 0.029: 100%|██████████| 1/1 [00:00<00:00, 37.60it/s]                                                                                      


🔊 Embeddings de AUDIO:
tensor([ 1.2100,  4.4502, -1.4657, -0.6059, -0.8751, -0.7740, -1.9395])

📝 Embeddings de TEXTO:
tensor([ 2.4287,  1.2394,  0.6221,  0.4580, -0.1798, -1.7292, -2.8392])


🎧 Segmento 2
----------------------------------------------------------------------


📌 Clasificación: Presentation
🗣️  Texto: We're about to begin.
⏱️  Intervalo de tiempo: 1.626s - 2.206s


...

rtf_avg: 0.019: 100%|██████████| 1/1 [00:00<00:00, 40.09it/s]                                                                                      


🔊 Embeddings de AUDIO:
tensor([ 1.4785,  4.9817, -3.1188, -0.4892, -2.6206, -0.7518,  0.5201])

📝 Embeddings de TEXTO:
tensor([ 3.8024,  0.7416, -0.5315, -0.7789, -0.8985, -1.0014, -1.3337])


🎧 Segmento 5
----------------------------------------------------------------------


📌 Clasificación: Presentation
🗣️  Texto: At this time, for opening remarks and introductions, I'd like to turn the call over to Nancy Paxton, Senior Director of Investor Relations.
⏱️  Intervalo de tiempo: 10.832s - 16.276s


rtf_avg: 0.008: 100%|██████████| 1/1 [00:00<00:00, 21.15it/s]                                                                                      


🔊 Embeddings de AUDIO:
tensor([ 4.7937,  9.8550, -5.7281, -2.2478, -1.8238, -2.0199, -2.8292])

📝 Embeddings de TEXTO:
tensor([ 3.7052,  0.4916, -0.3299, -0.4649, -0.7707, -1.0359, -1.5954])


